In [6]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pickle
import matlab.engine


import autograd.numpy as anp
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter
from pymoo.factory import get_termination

from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.core.population import Population

population=10
c_l = 0.002 * anp.ones(10)
c_h = 0.5 * anp.ones(10) 


In [7]:
#25000,5, 200
#36000,10, 338
#thrust=36000; vel_ship=10; rpm=338; 
#thrust=25000; vel_ship=5; rpm=200;
thrust=30600; vel_ship=1; rpm=50;
req=np.array([thrust,vel_ship,rpm])

In [30]:
eng = matlab.engine.start_matlab()

class MyProblem(Problem):
    def __init__(self, const_1=5, const_2=0.1):
        # define lower and upper bounds -  1d array with length equal to number of variable
        c_l = 0.002 * anp.ones(10)
        c_h = 0.5 * anp.ones(10)
        xl=np.append(np.array([0.01]),c_l); xu=np.append(np.array([10]),c_h)
        print('xl is:',xl,'xh is:',xu)
        super().__init__(n_var=11, n_obj=1, n_constr=0, xl=xl, xu=xu, evaluation_of="auto")
 

    def _evaluate(self, x, out, *args, **kwargs):
        f= self.ext_func(x)
        out["F"] = np.array(f).reshape(-1,1)
        
    
    def ext_func(self,x,*args,**kwargs) :
       eval=[];
       #print('X in eval is:',x)
       for i in range(len(x)):
         #print('*****************')
         test_d= np.append(req,x[i])
         #print('Design is:',test_d)
         tf = eng.OpenProp_eval(matlab.double(test_d.tolist()))
         eval.append(-1*tf)
         #print('-> Eval is:',eval)
       return eval


# Execute the optimizer 
problem = MyProblem()
termination = get_termination("n_gen", 50)
algorithm = GA(pop_size=10,
    eliminate_duplicates=True,save_history=True)
res = minimize(problem,
               algorithm,
               termination,
               seed=10,
               verbose=True,save_history=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))
eng.quit()

data= extract_best(res)
filename='./data/GA_th'+str(thrust)+'_vs'+str(vel_ship)+'_rpm'+str(rpm)+'.csv'
np.savetxt(filename, data, delimiter=",")
#print(res.history)

xl is: [0.01  0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002] xh is: [10.   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5]
n_gen |  n_eval |     fopt     |     favg    
    1 |      10 | -4.23982E-01 | -2.42420E-01
    2 |      20 | -4.37379E-01 | -4.04144E-01
    3 |      30 | -4.39320E-01 | -4.28435E-01
    4 |      40 | -4.40521E-01 | -4.36731E-01
    5 |      50 | -4.45247E-01 | -4.40582E-01
    6 |      60 | -4.46747E-01 | -4.43464E-01
    7 |      70 | -4.49650E-01 | -4.46260E-01
    8 |      80 | -4.51551E-01 | -4.47847E-01
    9 |      90 | -4.51553E-01 | -4.49449E-01
   10 |     100 | -4.52640E-01 | -4.51019E-01
   11 |     110 | -4.53728E-01 | -4.52207E-01
   12 |     120 | -4.55472E-01 | -4.53405E-01
   13 |     130 | -4.98535E-01 | -4.63019E-01
   14 |     140 | -5.00703E-01 | -4.81131E-01
   15 |     150 | -5.06276E-01 | -4.96773E-01
   16 |     160 | -5.08044E-01 | -5.01173E-01
   17 |     170 | -5.08846E-01 | -5.03881E-01
   18 |     180 | -5.10538E-01 | 

In [31]:
def extract_best(res):
 data_history= res.history
 flag=0
 for d in data_history:
    pop = d.pop
    y_out= pop.get("F")
    X= pop.get("X")
    #print(pop.get("F"))
    index= np.argmin(y_out) 
    #print(('X index:',X[index]))
    #print('y index:',y_out[index])
    z=np.append(X[index],y_out[index]).reshape(1,-1)
    #print('Z is:',z)
    if flag==0:
        data=z
        flag=1
    else: 
        data= np.concatenate((data,z),axis=0)
 print('Shape of data:',data.shape)
 return data

#data=extract_best(res)

In [32]:
def load_model():
    with open('rfmodel_prop_2l', 'rb') as f:
     model=pickle.load(f)
    return model

def prop_designer(req):
    design_req= req.reshape(1,-1)
    designed_prop = model.predict(design_req)
    predicted_design=np.atleast_2d(designed_prop[0][0:12])
    predicted_eff= designed_prop[0][-1]
    print("predicted design:",predicted_design)
    print("predicted accuracy:",predicted_eff*100, '%')
    return predicted_design,predicted_eff

In [33]:
model=load_model()
design,eff= prop_designer(req)
print('shape of design:', design.shape)

/home/hv/.local/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


predicted design: [[2.32299786e-01 2.10834990e-01 3.15374940e-01 2.26216410e-01
  2.06395850e-01 2.24653292e-01 2.56076360e-01 1.71370650e-01
  2.80062161e-01 2.00000000e-03 4.28075200e+00 7.27730000e-01]]
predicted accuracy: 64.13824999999999 %
shape of design: (1, 12)


/home/hv/.local/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [34]:
eng = matlab.engine.start_matlab()
mod_design= np.append(design[0][10],design[0][0:10])
req_design= np.append(req,mod_design)
print(req_design)
tf = eng.OpenProp_eval(matlab.double(req_design.tolist()))
print('GT eff is:',tf)
eng.quit()

[3.06000000e+04 1.00000000e+00 5.00000000e+01 4.28075200e+00
 2.32299786e-01 2.10834990e-01 3.15374940e-01 2.26216410e-01
 2.06395850e-01 2.24653292e-01 2.56076360e-01 1.71370650e-01
 2.80062161e-01 2.00000000e-03]
GT eff is: 0.4604625772424173


# Using seed as starting point 

In [ ]:
eng = matlab.engine.start_matlab()
population=10
problem = MyProblem()
c_l = 0.002 * anp.ones(10)
c_h = 0.5 * anp.ones(10)
x_l=np.append(np.array([0.01]),c_l); x_h=np.append(np.array([10]),c_h)
print('xl is:',x_l,'xh is:',x_h)


def create_intialseed(design,population,x_h,x_l):
   print('design from RF is:',design)
   num_design= design.shape[0]
   X_rand= np.random.random(size=(population-num_design,11))*(x_h-x_l)+ x_l 
   X= np.atleast_2d(np.append(design[0][10],design[0][0:10]))
   X= np.concatenate((X,X_rand),axis=0)
   return X


X= create_intialseed(design,population,x_h,x_l)
#X=np.atleast_2d(design[0][0:11])
#X= np.atleast_2d(np.append(design[0][10],design[0][0:10]))
#print('X is:',X)
termination = get_termination("n_gen", 50)

algorithm = GA(
    pop_size=population,
    eliminate_duplicates=True,sampling = X)

res = minimize(problem,
               algorithm,
               termination,
               seed=10,
               verbose=True,save_history=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))
data= extract_best(res)
filename='./data/SAO_th'+str(thrust)+'_vs'+str(vel_ship)+'_rpm'+str(rpm)+'.csv'
np.savetxt(filename, data, delimiter=",")

xl is: [0.01  0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002] xh is: [10.   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5]
xl is: [0.01  0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002] xh is: [10.   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5]
design from RF is: [[2.32299786e-01 2.10834990e-01 3.15374940e-01 2.26216410e-01
  2.06395850e-01 2.24653292e-01 2.56076360e-01 1.71370650e-01
  2.80062161e-01 2.00000000e-03 4.28075200e+00 7.27730000e-01]]


In [ ]:
eng = matlab.engine.start_matlab()

tf = eng.OpenProp_eval(matlab.double(design))
#OpenProp: th,Vs,rpm,D,Dhub,Cd1,cd2,cd3,cd4,cd5,cd6,cd7,cd8,cd9,cd10,eff
print('Original accuracy:',tf*100,'%')  

In [ ]:
eng.quit()

In [ ]:
def derivative(f1,f2,delta=1e-2):
  output = (f1-f2)/(delta)
  return output

In [ ]:
print('design is:',design)
#tf = eng.OpenProp_eval(matlab.double(design))

baseline=eng.OpenProp_eval(matlab.double(design))
print('baseline is:',baseline)

max_eval_track=[]

def check_valid_design(design):
   #print('-->checking valid design')
   #which_row= np.where((design >= 0.8) | (design <= 0.01))
   #print('which row is:',which_row)
   #design = np.delete(design, np.where((design >= 0.8) | (design <= 0.01))[0], axis=0) 
   design = np.delete(design, np.where((design <= 0.001))[0], axis=0)  
   #print('Design are:',design)
   return design 

def take_rand_steps(design):
 max_step_size=-0.01; steps=[]
 dimension=[3,4,5,6,7,8,9,10,11];
 designs=np.tile(design, (len(dimension),1))
 #print('old design are :',designs)
 #print('dim start is:',dimension[0])
 for i in range(len(dimension)):
   rand_step=max_step_size*np.random.random_sample() 
   designs[i][dimension[0]+i]=designs[i][dimension[0]+i]+rand_step
 #print('Modified designs are:',designs)
 return designs

def eval_designs(new_designs):
 eval=[];
 for i in range(len(new_designs)):
    #print(new_designs[i].tolist())
    tf = eng.OpenProp_eval(matlab.double(new_designs[i].tolist()))
    eval.append(tf)
 #print(eval)
 maxpos = eval.index(max(eval)) 
 print(eval[maxpos])
 max_eval_track.append(eval[maxpos])
 return new_designs[maxpos], eval[maxpos]
    

In [ ]:
last_best_design=design
for i in range(1000):
 new_designs=take_rand_steps(design)
 new_designs=check_valid_design(new_designs)
 if new_designs.shape[0]>0:
  design_f,eff = eval_designs(new_designs)
  if eff>baseline: 
        baseline=eff
        design=design_f
  else: 
    print('Didnt find better design')
    pass
    
 else:
  print('optimal local solution found')
  break

In [ ]:
plt.plot(max_eval_track)

In [219]:
design

array([2.50000000e+04, 5.00000000e+00, 2.00000000e+02, 2.22252619e+00,
       1.01279800e-03, 1.00379263e-03, 1.01253787e-03, 1.00064102e-03,
       1.01171081e-03, 1.01700175e-03, 1.00984704e-03, 1.00507121e-03,
       2.55769750e-01, 2.67250635e-01, 2.00000000e-03])